# Awk

[GNU Gawk User's Manual](https://www.gnu.org/software/gawk/manual/gawk.html)

[Awk on Wikipedia](https://en.wikipedia.org/wiki/AWK)

##### Tutorial Derived from the following:
http://linuxcommand.org/lc3_adv_awk.php

## History

The AWK programming language is truly one of the classic tools used in Unix. It dates back to the very earliest days of the Unix tradition. It was originally developed in the late 1970's at Bell Telephone Laboratories by Alfred Aho, Peter Weinberger, and Brian Kernighan. The name "AWK" comes from the last names of the three authors. It underwent major improvements in 1985 with the release of nawk or "new awk." It is that version that we still use today, though it is usually just called awk.

## So, What's It Good For?

Though AWK is fairly general purpose, it is really designed to create filters, that is, programs that accept standard input, transform data, and send it to standard output. In particular, AWK is very good at processing columnar data. This makes it a good choice for developing report generators, and tools that are used to re-format data. Since it has strong regular expression support, it's good for very small text extraction and reformatting problems, too. Like sed, many AWK programs are just one line long.

In recent years, AWK has fallen out of fashion, being supplanted by other, newer, interpreted languages such as Perl and python, but AWK still has some advantages:

 - It's easy to learn. The language is not overly complex and has a syntax much like the C programming language, so learning it will be useful in the future when we study other languages and tools.

 - It really excels at a solving certain types of problems.

## How It Works

 - Operates on a single `record` at a time
 
 - A `record` is a single line of text terminate by a newline character `\n`
 
 - Each `record` is separated into `fields`, which by default are separated by white space
 
 - Fields are given numeric variables: `$1`, `$2`, `$3`, `$4`, `$5`.....and so on and so on.
 
 - The field `$0` is special and representes the entire `record`.

In [1]:
awk --version

GNU Awk 4.0.2
Copyright (C) 1989, 1991-2012 Free Software Foundation.

This program is free software; you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation; either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program. If not, see http://www.gnu.org/licenses/.


### The first example

In [2]:
ls -l /usr/bin | awk '{print $0}'

total 389972
-rwxr-xr-x   1 root root       41496 Jun 30  2016 [
-rwxr-xr-x   1 root root      107856 Mar  2  2017 a2p
-rwxr-xr-x   1 root root       11248 May 30  2017 abrt-action-analyze-backtrace
-rwxr-xr-x   1 root root       15328 May 30  2017 abrt-action-analyze-c
-rwxr-xr-x   1 root root        1345 May 30  2017 abrt-action-analyze-ccpp-local
-rwxr-xr-x   1 root root        6821 May 30  2017 abrt-action-analyze-core
-rwxr-xr-x   1 root root       11224 May 30  2017 abrt-action-analyze-oops
-rwxr-xr-x   1 root root       11224 May 30  2017 abrt-action-analyze-python
-rwxr-xr-x   1 root root        2814 May 30  2017 abrt-action-analyze-vmcore
-rwxr-xr-x   1 root root        1348 May 30  2017 abrt-action-analyze-vulnerability
-rwxr-xr-x   1 root root       11264 May 30  2017 abrt-action-analyze-xorg
-rwxr-xr-x   1 root root        5002 May 30  2017 abrt-action-check-oops-for-hw-error
-rwxr-xr-x   1 root root       11248 May 30  2017 abrt-action-generate-backtrace
-rwxr-xr-x   1 roo

lrwxrwxrwx   1 root root           6 Jun 26  2017 bzcmp -> bzdiff
-rwxr-xr-x   1 root root        2128 Jul 31  2014 bzdiff
-rwxr-xr-x   1 root root        1677 Jul 31  2014 bzgrep
-rwxr-xr-x   1 root root       36752 Jul 31  2014 bzip2
-rwxr-xr-x   1 root root       15608 Jul 31  2014 bzip2recover
lrwxrwxrwx   1 root root           6 Jun 26  2017 bzless -> bzmore
-rwxr-xr-x   1 root root        1259 Jul 31  2014 bzmore
-rwxr-xr-x   4 root root      772704 Jun  2  2017 c++
-rwxr-xr-x   2 root root       36607 Mar  2  2017 c2ph
-rwxr-xr-x   1 root root         224 Jun  2  2017 c89
-rwxr-xr-x   1 root root         215 Jun  2  2017 c99
-rwxr-xr-x   1 root root       71600 Apr 19  2017 cairo-sphinx
-rwxr-xr-x   1 root root       33616 Jul 12  2018 cal
-rwxr-xr-x   1 root root        1638 May  5  2017 ca-legacy
-rwxr-xr-x   1 root root       25112 Jan 26  2014 calibrate_ppa
-rwxr-xr-x   1 root root       15592 Jan 26  2014 canberra-boot
-rwxr-xr-x   1 root root       11528 Jan 26  2014 canbe

-rwxr-xr-x   1 root root       10520 Aug 19  2014 db47_checkpoint
-rwxr-xr-x   1 root root       22784 Aug 19  2014 db47_codegen
-rwxr-xr-x   1 root root       10528 Aug 19  2014 db47_deadlock
-rwxr-xr-x   1 root root       14592 Aug 19  2014 db47_dump
-rwxr-xr-x   1 root root       18816 Aug 19  2014 db47_hotbackup
-rwxr-xr-x   1 root root       26960 Aug 19  2014 db47_load
-rwxr-xr-x   1 root root       64304 Aug 19  2014 db47_printlog
-rwxr-xr-x   1 root root       10520 Aug 19  2014 db47_recover
-rwxr-xr-x   1 root root       14560 Aug 19  2014 db47_stat
-rwxr-xr-x   1 root root       10472 Aug 19  2014 db47_upgrade
-rwxr-xr-x   1 root root       10464 Aug 19  2014 db47_verify
-rwxr-xr-x   1 root root       11488 Mar 20  2017 db_archive
-rwxr-xr-x   1 root root       11576 Mar 20  2017 db_checkpoint
-rwxr-xr-x   1 root root       15672 Mar 20  2017 db_deadlock
-rwxr-xr-x   1 root root       15688 Mar 20  2017 db_dump
-rwxr-xr-x   1 root root       65992 Mar 20  2017 db_dump185
-rwx

-rwxr-xr-x   1 root root       20064 May 30  2017 eu-strings
-rwxr-xr-x   1 root root       49800 May 30  2017 eu-strip
-rwxr-xr-x   1 root root       45272 May 30  2017 eu-unstrip
-rwxr-xr-x   1 root root      428504 Jul 17  2017 evince
-rwxr-xr-x   1 root root       50976 Jul 17  2017 evince-previewer
-rwxr-xr-x   1 root root       15824 Jul 17  2017 evince-thumbnailer
lrwxrwxrwx   1 root root           2 Sep 26  2017 ex -> vi
-rwxr-xr-x   1 root root       15848 Jan 24  2014 exempi
-rwxr-xr-x   1 root root       33240 Jun 30  2016 expand
-rwxr-xr-x   1 root root       11352 May 20  2015 expect
-rwxr-xr-x   1 root root       37432 Jun 30  2016 expr
-rwxr-xr-x   1 root root        1177 Apr 25  2016 extlookup2hiera
-rwxr-xr-x   1 root root         678 Aug  8  2014 f2py
lrwxrwxrwx   1 root root           4 Jun 29  2017 f2py.numpy -> f2py
lrwxrwxrwx   1 root root           8 Sep 26  2017 f95 -> gfortran
-rwxr-xr-x   1 root root         593 Jan 26  2016 facter
-rwxr-xr-x   1 root root    

-rwxr-xr-x   1 root root       75240 Nov 10  2017 gio
-rwxr-xr-x   1 root root       11600 Nov 10  2017 gio-querymodules-64
-rwxr-xr-x   1 root root        2250 Feb  7  2017 g-ir-annotation-tool
-rwxr-xr-x   1 root root      176560 Feb  7  2017 g-ir-compiler
-rwxr-xr-x   1 root root        2229 Feb  7  2017 g-ir-doc-tool
-rwxr-xr-x   1 root root       37280 Feb  7  2017 g-ir-generate
-rwxr-xr-x   1 root root       11392 Feb  7  2017 g-ir-inspect
-rwxr-xr-x   1 root root        2241 Feb  7  2017 g-ir-scanner
-rwxr-xr-x 112 root root     1515056 Aug 11  2017 git
-rwxr-xr-x 112 root root     1515056 Aug 11  2017 git-receive-pack
-rwxr-xr-x   2 root root      730912 Aug 11  2017 git-shell
-rwxr-xr-x 112 root root     1515056 Aug 11  2017 git-upload-archive
-rwxr-xr-x   2 root root      806160 Aug 11  2017 git-upload-pack
lrwxrwxrwx   1 root root          11 Sep 26  2017 gjs -> gjs-console
-rwxr-xr-x   1 root root       11792 Feb  9  2017 gjs-console
-rwxr-xr-x   1 root root       11680 Mar

...

-rwxr-xr-x   1 root root       43522 Apr 14  2017 remunge
-rwxr-xr-x   1 root root       11600 Jul 12  2018 rename
-rwxr-xr-x   1 root root       11552 Jul 12  2018 renice
-rwxr-xr-x   1 root root       11699 Mar 21  2017 repoclosure
-rwxr-xr-x   1 root root       14301 Mar 21  2017 repodiff
-rwxr-xr-x   1 root root        4191 Mar 21  2017 repo-graph
-rwxr-xr-x   1 root root        7043 Mar 21  2017 repomanage
-rwxr-xr-x   1 root root       57013 Mar 21  2017 repoquery
-rwxr-xr-x   1 root root       10315 Mar 21  2017 repo-rss
-rwxr-xr-x   1 root root       28720 Feb 21  2017 report-cli
-rwxr-xr-x   1 root root       49576 Feb 21  2017 reporter-bugzilla
-rwxr-xr-x   1 root root       15808 Feb 21  2017 reporter-mailx
-rwxr-xr-x   1 root root       41424 Feb 21  2017 reporter-rhtsupport
-rwxr-xr-x   1 root root       15648 Feb 21  2017 reporter-upload
-rwxr-xr-x   1 root root       19752 Feb 21  2017 reporter-ureport
-rwxr-xr-x   1 root root      130432 Feb 21  2017 report-gtk
-rwxr-xr

-rwxr-xr-x   1 root root      145512 Nov 24  2017 sftp
lrwxrwxrwx   1 root root           6 Nov  8  2016 sg -> newgrp
lrwxrwxrwx   1 root root           4 Sep 26  2017 sh -> bash
-rwxr-xr-x   1 root root       37440 Jun 30  2016 sha1sum
-rwxr-xr-x   1 root root       41600 Jun 30  2016 sha224sum
-rwxr-xr-x   1 root root       41600 Jun 30  2016 sha256sum
-rwxr-xr-x   1 root root       41600 Jun 30  2016 sha384sum
-rwxr-xr-x   1 root root       41600 Jun 30  2016 sha512sum
-rwxr-xr-x   1 root root     1518976 Feb 10  2017 shcomp
-rwxr-xr-x   1 root root       10371 Mar 21  2017 show-changed-rco
-rwxr-xr-x   1 root root       15832 Feb 22  2017 showconsolefont
-rwxr-xr-x   1 root root       16572 Mar 21  2017 show-installed
-rwxr-xr-x   1 root root       15608 Feb 22  2017 showkey
-rwxr-xr-x   1 root root        7248 Feb  6  2017 showrgb
-rwxr-xr-x   1 root root       54216 Jun 30  2016 shred
-rwxr-xr-x   1 root root       50320 Jun 30  2016 shuf
-rwxr-xr-x   1 root root      172072 Apr 

-rwxr-xr-x   1 root root       29664 Mar 27  2017 teamdctl
-rwxr-xr-x   1 root root       19568 Mar 27  2017 teamnl
-rwxr-xr-x   1 root root       33160 Jun 30  2016 tee
-rwxr-xr-x   1 root root       13960 Jan 27  2014 telepathy-gabble-xmpp-console
-rwxr-xr-x   1 root root      101776 Apr 19  2017 telnet
-rwxr-xr-x   1 root root       37328 Jun 30  2016 test
-rwxr-xr-x   1 root root       30488 Jan 24  2014 testgdbm
-rwxr-xr-x   1 root root       15776 Dec 28  2013 testlibraw
-rwxr-xr-x   1 root root       31760 Sep 14  2017 testparm
-rwxr-xr-x   1 root root        6843 Nov  6  2000 test_ppa
-rwxr-xr-x   1 root root          74 Mar  6  2017 testrb
-rwxr-xr-x   1 root root       52045 Jan 27  2014 texi2any
-rwxr-xr-x   1 root root        6465 Jan 24  2014 text2wave
-rwxr-xr-x   1 root root       32608 Mar  2  2016 tftp
-rwxr-xr-x   1 root root        2295 Jan 26  2014 tgz
-rwxr-xr-x   1 root root       65800 Aug 15  2017 tic
-rwxr-xr-x   1 root root       15920 Jan 27  2014 time
-rwxr-

-rwxr-xr-x   1 root root        1117 Aug 21  2014 vimdot
-rwxr-xr-x   1 root root        2084 Jan 20  2017 vimtutor
-rwxr-xr-x   1 root root      315704 Mar 27  2017 vinagre
-rwxr-xr-x   1 root root      636584 Aug 22  2017 virsh
-rwxr-xr-x   1 root root       28008 Aug 22  2017 virt-host-validate
-rwxr-xr-x   1 root root        9801 Aug 22  2017 virt-pki-validate
-rwxr-xr-x   1 root root        2279 Aug 22  2017 virt-xml-validate
-rwxr-xr-x   1 root root       16104 Feb 22  2017 vlock
-rwxr-xr-x   1 root root       49944 Mar 28  2017 vmhgfs-fuse
-rwxr-xr-x   1 root root       11560 Feb  1  2017 vmmouse_detect
-rwxr-xr-x   1 root root       32224 Jun  1  2017 vmstat
-rwxr-xr-x   1 root root        9727 Mar 28  2017 vm-support
-rwxr-xr-x   1 root root       48896 Mar 28  2017 vmtoolsd
-rwxr-xr-x   1 root root       11328 Mar 28  2017 vmware-checkvm
-rwxr-xr-x   1 root root       32536 Mar 28  2017 vmware-guestproxycerttool
-rwxr-xr-x   1 root root       11464 Mar 28  2017 vmware-hgfscli

lrwxrwxrwx   1 root root           6 Nov  8  2016 xzegrep -> xzgrep
lrwxrwxrwx   1 root root           6 Nov  8  2016 xzfgrep -> xzgrep
-rwxr-xr-x   1 root root        5628 Mar  4  2016 xzgrep
-rwxr-xr-x   1 root root        1802 Mar  4  2016 xzless
-rwxr-xr-x   1 root root        2161 Mar  4  2016 xzmore
-rwxr-xr-x   1 root root       55008 Mar  7  2017 yelp
-rwxr-xr-x   1 root root       28984 Jun 30  2016 yes
lrwxrwxrwx   1 root root           8 Nov  8  2016 ypdomainname -> hostname
-rwxr-xr-x   1 root root         801 Mar 27  2017 yum
-rwxr-xr-x   1 root root        9429 Mar 21  2017 yum-builddep
-rwxr-xr-x   1 root root        9039 Mar 21  2017 yum-config-manager
-rwxr-xr-x   1 root root        7609 Mar 21  2017 yum-debug-dump
-rwxr-xr-x   1 root root        7903 Mar 21  2017 yum-debug-restore
-rwxr-xr-x   1 root root       10999 Mar 21  2017 yumdownloader
-rwxr-xr-x   1 root root       11031 Mar 21  2017 yum-groups-manager
-rwxr-xr-x   1 root root        1941 Feb 28  2017 zcat
-r

### Breaking down the first example

- The awk program is contained within the single quotation marks `'{print $0}'`

- Quotation marks prevent the shell from expanding the `$` symbol

- The output of `ls -l /usr/bin` is piped into our program line by line, i.e. record by record.

- `{print $0}` prints out the entire record.

In [3]:
# print the first field

ls -l /usr/bin | awk '{print $1}'

total
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
lrwxrwxrwx
-rwxr-xr-x
-rwxr-xr-x
lrwxrwxrwx
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwsr-xr-x
lrwxrwxrwx
lrwxrwxrwx
-rwxr-xr-x
-rwxr-xr-x
lrwxrwxrwx
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
lrwxrwxrwx
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
lrwxrwxrwx
-rwxr-xr-x
-rwx

-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
lrwxrwxrwx
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
lrwxrwxrwx
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
lrwxrwxrwx

lrwxrwxrwx
lrwxrwxrwx
lrwxrwxrwx
-rwxr-xr-x
-rwxr-xr-x
lrwxrwxrwx
-rwxr-xr-x
lrwxrwxrwx
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
lrwxrwxrwx
lrwxrwxrwx
-rwxr-xr-x
-rwxr-xr-x
lrwxrwxrwx
lrwxrwxrwx
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x
-rwxr-xr-x

In [4]:
# print the ninth field

ls -l /usr/bin | awk '{print $9}'


[
a2p
abrt-action-analyze-backtrace
abrt-action-analyze-c
abrt-action-analyze-ccpp-local
abrt-action-analyze-core
abrt-action-analyze-oops
abrt-action-analyze-python
abrt-action-analyze-vmcore
abrt-action-analyze-vulnerability
abrt-action-analyze-xorg
abrt-action-check-oops-for-hw-error
abrt-action-generate-backtrace
abrt-action-generate-core-backtrace
abrt-action-install-debuginfo
abrt-action-list-dsos
abrt-action-notify
abrt-action-perform-ccpp-analysis
abrt-action-save-kernel-data
abrt-action-save-package-data
abrt-action-trim-files
abrt-applet
abrt-cli
abrt-dump-oops
abrt-dump-xorg
abrt-handle-upload
abrt-merge-pstoreoops
abrt-watch-log
abs2rel
ac
aclocal
aclocal-1.13
aconnect
acyclic
addr2line
alias
alsabat
alsa-delay
alsaloop
alsamixer
alsatplg
alsaucm
alsaunmute
amidi
amixer
amuFormat.sh
anaconda-cleanup
anaconda-disable-nm-ibft-plugin
analog
animate
aplay
aplaymidi
applydeltarpm
apropos
ar
arch
arecord
arecordmidi
as
asadmin
ascii-xfr
aseqdump
aseqnet
aserver
at
atq
atrm
attr


host
hostid
hostname
hostnamectl
hpcups-update-ppds
hpijs
htop
hunspell
i2c
i386
ib_acme
ib_atomic_bw
ib_atomic_lat
ibcongest
ibdev2netdev
ibdiagm.sh
ibdiagnet
ibdiagnet_csv2xml.py
ibdiagpath
ibdmchk
ibdmtr
ibdump
ibgenperm
ibnetsplit
ibnlparse
ib_read_bw
ib_read_lat
ib_send_bw
ib_send_lat
ibsim
ibtopodiff
ibus
ibus-daemon
ibus-setup
ibus-table-createdb
ibv_asyncwatch
ibv_cc_pingpong
ibv_dcini
ibv_dctgt
ibv_devices
ibv_devinfo
ibv_intf
ibv_polldcinfo
ibv_rc_pingpong
ibv_srq_pingpong
ibv_task_pingpong
ibv_uc_pingpong
ibv_ud_pingpong
ibv_umr
ibv_xsrq_pingpong
ib_write_bw
ib_write_lat
iceauth
iconv
id
identify
idiag-socket-details
idn
iecset
ifnames
igawk
ijsgutenprint.5.2
implantisomd5
import
imsettings-info
imsettings-list
imsettings-reload
imsettings-switch
index_dump
info
infocmp
infokey
infotocap
inputattach
install
install-printerdriver
instance_events
instmodsh
instperf
ionice
iostat
ipcalc
ipcmk
ipcrm
ipcs
iperf
ipmicmd
ipmilan
ipmish
ipmi_ui
iptables-xml
iptc
irb
isdv4-serial-inp

raw_ethernet_lat
rb
rcc
rcopy
rct
rctest
rdate
rdfproc
rdma_client
rdma_server
rdma_xclient
rdma_xserver
rdoc
read
readcd
readelf
readlink
readmult
readom
realpath
rebuild-jar-repository
rec
recode-sr-latin
red
redhat-access-insights
redhat_lsb_init
redhat-support-tool
redland-db-upgrade
remunge
rename
renice
repoclosure
repodiff
repo-graph
repomanage
repoquery
repo-rss
report-cli
reporter-bugzilla
reporter-mailx
reporter-rhtsupport
reporter-upload
reporter-ureport
report-gtk
reposync
repotrack
reset
resize
resizecons
rev
rfcomm
rftp
rhino
rhn_register
rhsmcertd
rhsm-debug
ri
riostream
rksh
rlogin-cwd
rm
rmail
rmail.postfix
rmcp_ping
rmdir
rmid
rmiregistry
rnano
rngtest
roqet
route53
rpcgen
rping
rpm
rpm2cpio
rpmbuild
rpmdb
rpmdumpheader
rpmgraph
rpmkeys
rpmquery
rpmspec
rpmverify
rstream
rsync
rsyslog-recover-qi.pl
ruby
runcon
run-parts
run_perftest_loopback
run_perftest_multi_devices
runscript
rvi
rview
rvim
rx
rxe_cfg
rz
s2p
s3put
sadf
sandbox
sane-find-scanner
sar
sas_disk_blink
sa

Most lines returned by `ls -l /usr/bin` have nine fields separated by whitespace.

But some files in `/usr/bin` are symbolically links to other files, which show up in subsequent fields.

Let's find these by print all records where the number of fields exceeds 9.

In [5]:
ls -l /usr/bin | awk 'NF > 9 {print $0}'

lrwxrwxrwx   1 root root           6 Nov  8  2016 apropos -> whatis
lrwxrwxrwx   1 root root           5 Sep 26  2017 arecord -> aplay
lrwxrwxrwx   1 root root           2 Jun 26  2017 atq -> at
lrwxrwxrwx   1 root root           2 Jun 26  2017 atrm -> at
lrwxrwxrwx   1 root root          11 Sep 26  2017 audit2why -> audit2allow
lrwxrwxrwx   1 root root           4 Jul  5  2017 awk -> gawk
lrwxrwxrwx   1 root root          10 Sep 26  2017 bashbug -> bashbug-64
lrwxrwxrwx   1 root root           5 Jun 26  2017 bunzip2 -> bzip2
lrwxrwxrwx   1 root root           5 Jun 26  2017 bzcat -> bzip2
lrwxrwxrwx   1 root root           6 Jun 26  2017 bzcmp -> bzdiff
lrwxrwxrwx   1 root root           6 Jun 26  2017 bzless -> bzmore
lrwxrwxrwx   1 root root          30 Sep 26  2017 cancel -> /etc/alternatives/print-cancel
lrwxrwxrwx   1 root root           3 Sep 26  2017 captoinfo -> tic
lrwxrwxrwx   1 root root           3 Sep 26  2017 cc -> gcc
lrwxrwxrwx   1 root root          26 Jun 26  2017 cd

lrwxrwxrwx   1 root root          32 Nov  8  2016 newaliases -> /etc/alternatives/mta-newaliases
lrwxrwxrwx   1 root root          31 Nov  8  2016 newaliases.postfix -> ../../usr/sbin/sendmail.postfix
lrwxrwxrwx   1 root root           8 Nov  8  2016 nisdomainname -> hostname
lrwxrwxrwx   1 root root           5 Sep 26  2017 nmtui-connect -> nmtui
lrwxrwxrwx   1 root root           5 Sep 26  2017 nmtui-edit -> nmtui
lrwxrwxrwx   1 root root           5 Sep 26  2017 nmtui-hostname -> nmtui
lrwxrwxrwx   1 root root          38 Sep 26  2017 npm -> ../lib/node_modules/npm/bin/npm-cli.js
lrwxrwxrwx   1 root root           6 Sep 26  2017 open -> openvt
lrwxrwxrwx   1 root root          11 Sep 26  2017 openoffice.org -> libreoffice
lrwxrwxrwx   1 root root          22 Sep 26  2017 orbd -> /etc/alternatives/orbd
lrwxrwxrwx   1 root root           3 Jun 26  2017 osage -> dot
lrwxrwxrwx   1 root root          25 Sep 26  2017 pack200 -> /etc/alternatives/pack200
lrwxrwxrwx   1 root root          

### Special patterns

Awk patterns can run in many forms, such as conditional expressions in the above example and regular expressions.

Two special patterns exist: `BEGIN` and `END`

- `BEGIN` carries out its action before the first record is read
- `END` carries out its action after the last record is read

### Special Pattern Example

##### Building a directory report

Let's employ `BEGIN` and `END` to build a directory report from `ls -l /usr/bin`. 

Steps:

1) Use `BEGIN` to make a header.

```
Directory Report
================
```

2) For each record that has more than nine fields, use this pattern: `{print $9, "is a symbolic link to", $NF}`

3) Create a footer at the very end of the report.
```
=============
End Of Report
```

In [6]:
# Print a directory report

ls -l /usr/bin | awk '
    BEGIN {
        print "Directory Report"
        print "================"
    }

    NF > 9 {
        print $9, "is a symbolic link to", $NF
    }

    END {
        print "============="
        print "End Of Report"
    }

'

Directory Report
apropos is a symbolic link to whatis
arecord is a symbolic link to aplay
atq is a symbolic link to at
atrm is a symbolic link to at
audit2why is a symbolic link to audit2allow
awk is a symbolic link to gawk
bashbug is a symbolic link to bashbug-64
bunzip2 is a symbolic link to bzip2
bzcat is a symbolic link to bzip2
bzcmp is a symbolic link to bzdiff
bzless is a symbolic link to bzmore
cancel is a symbolic link to /etc/alternatives/print-cancel
captoinfo is a symbolic link to tic
cc is a symbolic link to gcc
cdrecord is a symbolic link to /etc/alternatives/cdrecord
charmap is a symbolic link to gucharmap
circo is a symbolic link to dot
csh is a symbolic link to tcsh
desktop-file-edit is a symbolic link to desktop-file-install
dnsdomainname is a symbolic link to hostname
domainname is a symbolic link to hostname
dot2gxl is a symbolic link to gxl2gv
dvdrecord is a symbolic link to /etc/alternatives/cdrecord-dvdrecord
emacs is a symbolic link to /etc/alternatives/emacs
et

vgllogin is a symbolic link to /opt/VirtualGL/bin/vgllogin
vglrun is a symbolic link to /opt/VirtualGL/bin/vglrun
vglserver_config is a symbolic link to /opt/VirtualGL/bin/vglserver_config
view is a symbolic link to vi
vimdiff is a symbolic link to vim
wish is a symbolic link to wish8.5
X is a symbolic link to Xorg
x86_64 is a symbolic link to setarch
xzcat is a symbolic link to xz
xzcmp is a symbolic link to xzdiff
xzegrep is a symbolic link to xzgrep
xzfgrep is a symbolic link to xzgrep
ypdomainname is a symbolic link to hostname
zsoelim is a symbolic link to soelim
End Of Report


### Relational expressions

Relational expressions are used to test fields for values. For example, we can test for equivalence:

`$1 == "Fedora"`

or for relations such as:

`$3 >= 50`

It is also possible to perform calculations like:

`$1 * $2 < 100`

### pattern logical-operator pattern

It is possible to combine patterns together using the logical operators || and &&, meaning OR and AND, respectively.

For example, to test a record to see if the first field is greater than 100 and the last field is "Debit":

`$1 > 100 && $NF == "Debit"`

## Built-in Variables

### `FS`

##### Field separator

By default, awk automatically separates fields by whitespace. The field separator `FS` variable can be set for other field separators.

```
BEGIN { FS = ":" }
{ print $1, $5 }
```

### `RS`

##### Record separator

Records are automatically separated by the newline character `\n`, but can also be specified by the record separator variable `RS`. For example, if records are separated by empty lines, then we modify our `RS` to be:

```
BEGIN { FS = "\n"; RS = "" }
```

### `NF`

##### Number of fields

This variable updates each time a record is read. We can easily access the last field in the record by referring to `$NF`.

### `OFS`

##### Output field separator

This string is used to separate fields when printing output. The default is a single space. Setting this can be handy when reformatting data, e.g.,

In [7]:
ls -l /usr/bin | awk 'BEGIN {OFS = ","} NF == 9 {print $1,$2,$3,$4,$5,$6,$7,$8,$9}'

-rwxr-xr-x,1,root,root,41496,Jun,30,2016,[
-rwxr-xr-x,1,root,root,107856,Mar,2,2017,a2p
-rwxr-xr-x,1,root,root,11248,May,30,2017,abrt-action-analyze-backtrace
-rwxr-xr-x,1,root,root,15328,May,30,2017,abrt-action-analyze-c
-rwxr-xr-x,1,root,root,1345,May,30,2017,abrt-action-analyze-ccpp-local
-rwxr-xr-x,1,root,root,6821,May,30,2017,abrt-action-analyze-core
-rwxr-xr-x,1,root,root,11224,May,30,2017,abrt-action-analyze-oops
-rwxr-xr-x,1,root,root,11224,May,30,2017,abrt-action-analyze-python
-rwxr-xr-x,1,root,root,2814,May,30,2017,abrt-action-analyze-vmcore
-rwxr-xr-x,1,root,root,1348,May,30,2017,abrt-action-analyze-vulnerability
-rwxr-xr-x,1,root,root,11264,May,30,2017,abrt-action-analyze-xorg
-rwxr-xr-x,1,root,root,5002,May,30,2017,abrt-action-check-oops-for-hw-error
-rwxr-xr-x,1,root,root,11248,May,30,2017,abrt-action-generate-backtrace
-rwxr-xr-x,1,root,root,11240,May,30,2017,abrt-action-generate-core-backtrace
-rwxr-xr-x,1,root,root,8341,May,30,2017,abrt-action-install-debuginfo
-rwxr-

-rwxr-xr-x,1,root,root,6280,Sep,9,2017,centrino-decode
-rwxr-xr-x,1,root,root,151024,Jun,15,2017,certtool
-rwxr-xr-x,1,root,root,171624,Apr,18,2018,certutil
-rwxr-xr-x,1,root,root,3442,Jan,27,2017,cfadmin
-rwxr-xr-x,1,root,root,28584,Aug,9,2017,c++filt
-rwxr-sr-x,1,root,cgred,15624,Mar,16,2017,cgclassify
-rwxr-xr-x,1,root,root,15760,Mar,16,2017,cgcreate
-rwxr-xr-x,1,root,root,15456,Mar,16,2017,cgdelete
-rwxr-sr-x,1,root,cgred,15592,Mar,16,2017,cgexec
-rwxr-xr-x,1,root,root,19624,Mar,16,2017,cgget
-rwxr-xr-x,1,root,root,15464,Mar,16,2017,cgset
-rwxr-xr-x,1,root,root,19792,Mar,16,2017,cgsnapshot
-rwxr-xr-x,1,root,root,15640,Sep,21,2017,chacl
-rwsr-xr-x,1,root,root,64232,Jun,28,2016,chage
-rwxr-xr-x,1,root,root,307,Jun,19,2015,chardetect
-rwxr-xr-x,1,root,root,11616,Apr,10,2018,chattr
-rwxr-xr-x,1,root,root,13430,May,26,2017,chcat
-rwxr-xr-x,1,root,root,62928,Jun,30,2016,chcon
-rwxr-xr-x,1,root,root,3328,Jul,2,2015,check-binary-files
-rwxr-xr-x,1,root,root,19888,Aug,26,2014,checkisomd5
-r

-rwxr-xr-x,1,root,root,112960,Jun,30,2016,du
-rwxr-xr-x,1,root,root,2280,Feb,27,2018,dump2psl.pl
-rwxr-xr-x,1,root,root,1702,Feb,27,2018,dump2slvl.pl
-rwxr-xr-x,1,root,root,11744,Dec,28,2013,dumpiso
-rwxr-xr-x,1,root,root,79360,Feb,22,2017,dumpkeys
-rwxr-xr-x,1,root,root,15552,Feb,28,2017,dumpsexp
-rwxr-xr-x,1,root,root,15704,Dec,28,2013,dvcont
-rwxr-xr-x,1,root,root,23912,Jan,24,2014,dvd-ram-control
-rwxr-xr-x,1,root,root,65080,Jan,24,2014,dvd+rw-booktype
-rwxr-xr-x,1,root,root,44640,Jan,24,2014,dvd+rw-format
-rwxr-xr-x,1,root,root,48528,Jan,24,2014,dvd+rw-mediainfo
-rwxr-xr-x,1,root,root,1026,Aug,22,2017,dvipdf
-rwxr-xr-x,1,root,root,3073704,Aug,9,2017,dwp
-rwxr-xr-x,1,root,root,166504,Jan,24,2014,dwz
-rwxr-xr-x,1,root,root,2131,Jan,27,2017,dynamodb_dump
-rwxr-xr-x,1,root,root,3501,Jan,27,2017,dynamodb_load
-rwxr-xr-x,1,root,root,320,Feb,15,2017,easy_install
-rwxr-xr-x,1,root,root,328,Feb,15,2017,easy_install-2.7
-rwxr-xr-x,1,root,root,45360,Sep,12,2017,ebrowse
-rwxr-xr-x,1,root,root

-rwxr-xr-x,1,root,root,28384,Jul,12,2017,gdmflexiserver
-rwxr-xr-x,1,root,root,15472,Jul,12,2017,gdm-screenshot
-rwxr-xr-x,1,root,root,11376,May,30,2017,gedit
-rwxr-xr-x,1,root,root,542,Mar,6,2017,gem
-rwxr-xr-x,1,root,root,23064,Jun,16,2017,gencat
-rwxr-xr-x,1,root,root,586,Mar,17,2017,gendiff
-rwxr-xr-x,1,root,root,521136,Sep,22,2014,genisoimage
-rwxr-xr-x,2,root,root,11544,Apr,18,2017,genl-ctrl-list
-rwxr-xr-x,1,root,root,15648,Jun,29,2016,geoiplookup
-rwxr-xr-x,1,root,root,11432,Jun,29,2016,geoiplookup6
-rwxr-xr-x,1,root,root,11496,Jun,29,2016,geoipupdate
-rwxr-xr-x,1,root,root,15059,Dec,29,2013,GET
-rwxr-xr-x,1,root,root,22848,Jun,16,2017,getconf
-rwxr-xr-x,1,root,root,27832,Jun,16,2017,getent
-rwxr-xr-x,1,root,root,24872,Sep,21,2017,getfacl
-rwxr-xr-x,1,root,root,20544,Sep,21,2017,getfattr
-rwxr-xr-x,1,root,root,11424,Feb,22,2017,getkeycodes
-rwxr-xr-x,1,root,root,11320,Jan,27,2014,get_module
-rwxr-xr-x,1,root,root,15832,Jul,12,2018,getopt
-rwxr-xr-x,1,root,root,31,Aug,3,2017,get

-rwxr-xr-x,1,root,root,32904,Mar,9,2017,gst-launch-1.0
-rwxr-xr-x,1,root,root,24240,Mar,9,2017,gst-stats-1.0
-rwxr-xr-x,1,root,root,16000,Jan,24,2014,gst-typefind
-rwxr-xr-x,1,root,root,15728,Jan,24,2014,gst-typefind-0.10
-rwxr-xr-x,1,root,root,15752,Mar,9,2017,gst-typefind-1.0
-rwxr-xr-x,1,root,root,16000,Jan,24,2014,gst-xmlinspect
-rwxr-xr-x,1,root,root,24072,Jan,24,2014,gst-xmlinspect-0.10
-rwxr-xr-x,1,root,root,16000,Jan,24,2014,gst-xmllaunch
-rwxr-xr-x,1,root,root,36976,Jan,24,2014,gst-xmllaunch-0.10
-rwxr-xr-x,1,root,root,24280,Nov,10,2017,gtester
-rwxr-xr-x,1,root,root,18759,Nov,10,2017,gtester-report
-rwxr-xr-x,1,root,root,15560,Jun,26,2017,gtf
-rwxr-xr-x,1,root,root,15848,May,30,2017,gtk-launch
-rwxr-xr-x,1,root,root,15656,Feb,8,2017,gtk-query-immodules-2.0-64
-rwxr-xr-x,1,root,root,15816,May,30,2017,gtk-query-immodules-3.0-64
-rwxr-xr-x,1,root,root,33224,May,30,2017,gtk-update-icon-cache
-rwxr-xr-x,1,root,root,76584,Mar,10,2017,gucharmap
-rwxr-xr-x,1,root,root,2253,Feb,28,201

...

-rwxr-xr-x,2,root,root,11400,Mar,2,2017,perl
-rwxr-xr-x,2,root,root,11400,Mar,2,2017,perl5.16.3
-rwxr-xr-x,2,root,root,43873,Mar,2,2017,perlbug
-rwxr-xr-x,1,root,root,203,Dec,29,2013,perldoc
-rwxr-xr-x,1,root,root,10772,Mar,2,2017,perlivp
-rwxr-xr-x,1,root,root,12185,May,9,2015,perlsh
-rwxr-xr-x,2,root,root,43873,Mar,2,2017,perlthanks
-rwxr-xr-x,1,root,root,498,Aug,22,2017,pf2afm
-rwxr-xr-x,1,root,root,516,Aug,22,2017,pfbtopfa
-rwxr-xr-x,1,root,root,1860,Aug,24,2016,pflags
-rwxr-xr-x,1,root,root,428704,Jun,13,2017,pgawk
-rwxr-xr-x,1,root,root,28336,Jun,1,2017,pgrep
-rwxr-xr-x,1,root,root,184720,Jan,29,2014,pic
-rwxr-xr-x,1,root,root,8177,Jan,26,2014,piconv
-rwxr-xr-x,1,root,root,66368,Mar,29,2017,pidstat
-rwxr-xr-x,1,root,root,2602,Mar,17,2016,pinentry
-rwxr-xr-x,1,root,root,50328,Mar,17,2016,pinentry-curses
-rwxr-xr-x,1,root,root,110728,Mar,17,2016,pinentry-gtk-2
-rwxr-xr-x,1,root,root,100872,Jan,26,2014,pinfo
-rwxr-xr-x,1,root,root,66168,May,22,2017,ping
-rwxr-xr-x,1,root,root,37448,

-rwxr-xr-x,1,root,root,28,Aug,3,2017,read
-rwxr-xr-x,1,root,root,493192,Aug,9,2017,readelf
-rwxr-xr-x,1,root,root,41800,Jun,30,2016,readlink
-rwxr-xr-x,1,root,root,800,Sep,22,2014,readmult
-rwxr-xr-x,1,root,root,99792,Sep,22,2014,readom
-rwxr-xr-x,1,root,root,62704,Jun,30,2016,realpath
-rwxr-xr-x,1,root,root,1803,Jul,2,2015,rebuild-jar-repository
-rwxr-xr-x,1,root,root,15648,Mar,15,2017,recode-sr-latin
-rwxr-xr-x,1,root,root,89,Jan,24,2014,red
-rwxr-xr-x,1,root,root,379,Mar,24,2017,redhat-access-insights
-rw-r--r--,1,root,root,576,Feb,28,2012,redhat_lsb_init
-rwxr-xr-x,1,root,root,12011,May,31,2017,redhat-support-tool
-rwxr-xr-x,1,root,root,11384,Jan,27,2014,redland-db-upgrade
-rwxr-xr-x,1,root,root,43522,Apr,14,2017,remunge
-rwxr-xr-x,1,root,root,11600,Jul,12,2018,rename
-rwxr-xr-x,1,root,root,11552,Jul,12,2018,renice
-rwxr-xr-x,1,root,root,11699,Mar,21,2017,repoclosure
-rwxr-xr-x,1,root,root,14301,Mar,21,2017,repodiff
-rwxr-xr-x,1,root,root,4191,Mar,21,2017,repo-graph
-rwxr-xr-x,1,ro

-rwxr-xr-x,1,root,root,117720,Jun,30,2016,sort
-rwxr-xr-x,1,root,root,4341,Jun,16,2017,sotruss
-rwxr-xr-x,1,root,root,15696,Jan,26,2014,sound_dump
-rwxr-xr-x,1,root,root,27264,Jan,27,2014,soundstretch
-rwxr-xr-x,1,root,root,70360,Jan,27,2014,sox
-rwxr-xr-x,1,root,root,331880,May,20,2015,spax
-rwxr-xr-x,1,root,root,218,Sep,10,2010,spd-conf
-rwxr-xr-x,1,root,root,20456,Jan,27,2014,spd-say
-rwxr-xr-x,1,root,root,15928,Jan,27,2014,spdsend
-rwxr-xr-x,1,root,root,32576,Mar,24,2017,speaker-test
-rwxr-xr-x,1,root,root,141784,Jan,27,2014,speech-dispatcher
-rwxr-xr-x,1,root,root,62520,Jul,8,2016,spice-vdagent
-rwxr-xr-x,1,root,root,18459,Mar,2,2017,splain
-rwxr-xr-x,1,root,root,71128,Jun,30,2016,split
-rwxr-xr-x,1,root,root,23152,Jun,16,2017,sprof
-rwxr-xr-x,1,root,root,56240,Jul,23,2015,sqlite3
-rwxr-xr-x,1,root,root,774736,Nov,24,2017,ssh
-rwxr-xr-x,1,root,root,356904,Nov,24,2017,ssh-add
---x--s--x,1,root,nobody,382328,Nov,24,2017,ssh-agent
-rwxr-xr-x,1,root,root,10469,Nov,24,2017,ssh-copy-id


-rwxr-xr-x,1,root,root,45784,Jun,30,2016,uniq
-rwxr-xr-x,1,root,root,4165,Aug,22,2017,unix-lpr.sh
-rwxr-xr-x,1,root,root,28984,Jun,30,2016,unlink
-rwxr-xr-x,1,root,root,34399,Apr,14,2017,unmunge
-rwxr-xr-x,1,root,root,370,Jun,19,2017,unopkg
-rwxr-xr-x,1,root,root,15904,Jul,12,2018,unshare
-rwxr-xr-x,2,root,root,185488,Apr,15,2016,unzip
-rwxr-xr-x,1,root,root,90728,Apr,15,2016,unzipsfx
-rwxr-xr-x,1,root,root,1054,May,5,2017,update-ca-trust
-rwxr-xr-x,1,root,root,48792,Feb,4,2016,updatedb
-rwxr-xr-x,1,root,root,20096,Feb,21,2017,update-desktop-database
-rwxr-xr-x,1,root,root,313,Feb,8,2017,update-gtk-immodules
-rwxr-xr-x,1,root,root,54080,Mar,22,2017,update-mime-database
-rwxr-xr-x,1,root,root,15408,Mar,10,2017,upower
-rwxr-xr-x,1,root,root,11480,Jun,1,2017,uptime
-rwxr-xr-x,1,root,root,12465,Jun,30,2016,urlgrabber
-rwxr-xr-x,1,root,root,4216,Jun,6,2013,usb-devices
-rwxr-xr-x,1,root,root,28384,Apr,2,2015,usbhid-dump
-rwxr-xr-x,1,root,root,33192,Jun,30,2016,users
-rwxr-xr-x,1,root,root,11

-rwxr-xr-x,1,root,root,15536,Mar,28,2017,xvinfo
-rwxr-xr-x,1,root,root,2491344,May,17,2017,Xvnc
-rwxr-xr-x,1,root,root,45144,Mar,28,2017,xwininfo
-rwxr-xr-x,1,root,root,14768,Jan,20,2017,xxd
-rwxr-xr-x,1,root,root,75240,Mar,4,2016,xz
-rwxr-xr-x,1,root,root,11448,Mar,4,2016,xzdec
-rwxr-xr-x,1,root,root,6632,Mar,4,2016,xzdiff
-rwxr-xr-x,1,root,root,5628,Mar,4,2016,xzgrep
-rwxr-xr-x,1,root,root,1802,Mar,4,2016,xzless
-rwxr-xr-x,1,root,root,2161,Mar,4,2016,xzmore
-rwxr-xr-x,1,root,root,55008,Mar,7,2017,yelp
-rwxr-xr-x,1,root,root,28984,Jun,30,2016,yes
-rwxr-xr-x,1,root,root,801,Mar,27,2017,yum
-rwxr-xr-x,1,root,root,9429,Mar,21,2017,yum-builddep
-rwxr-xr-x,1,root,root,9039,Mar,21,2017,yum-config-manager
-rwxr-xr-x,1,root,root,7609,Mar,21,2017,yum-debug-dump
-rwxr-xr-x,1,root,root,7903,Mar,21,2017,yum-debug-restore
-rwxr-xr-x,1,root,root,10999,Mar,21,2017,yumdownloader
-rwxr-xr-x,1,root,root,11031,Mar,21,2017,yum-groups-manager
-rwxr-xr-x,1,root,root,1941,Feb,28,2017,zcat
-rwxr-xr-x,1,root,

### `ORS`

##### Output record separator

This is the string used to separate records when printing output. The default is a newline character. We could use this variable to easily double-space a file by setting ORS to equal two newlines:

In [8]:
ls -l /usr/bin | awk 'BEGIN {ORS = "\n\n"} {print}'

total 389972

-rwxr-xr-x   1 root root       41496 Jun 30  2016 [

-rwxr-xr-x   1 root root      107856 Mar  2  2017 a2p

-rwxr-xr-x   1 root root       11248 May 30  2017 abrt-action-analyze-backtrace

-rwxr-xr-x   1 root root       15328 May 30  2017 abrt-action-analyze-c

-rwxr-xr-x   1 root root        1345 May 30  2017 abrt-action-analyze-ccpp-local

-rwxr-xr-x   1 root root        6821 May 30  2017 abrt-action-analyze-core

-rwxr-xr-x   1 root root       11224 May 30  2017 abrt-action-analyze-oops

-rwxr-xr-x   1 root root       11224 May 30  2017 abrt-action-analyze-python

-rwxr-xr-x   1 root root        2814 May 30  2017 abrt-action-analyze-vmcore

-rwxr-xr-x   1 root root        1348 May 30  2017 abrt-action-analyze-vulnerability

-rwxr-xr-x   1 root root       11264 May 30  2017 abrt-action-analyze-xorg

-rwxr-xr-x   1 root root        5002 May 30  2017 abrt-action-check-oops-for-hw-error

-rwxr-xr-x   1 root root       11248 May 30  2017 abrt-action-generate-backtrace

-rwx


-rwxr-xr-x   1 root root      362536 Aug 16  2017 busctl

lrwxrwxrwx   1 root root           5 Jun 26  2017 bzcat -> bzip2

lrwxrwxrwx   1 root root           6 Jun 26  2017 bzcmp -> bzdiff

-rwxr-xr-x   1 root root        2128 Jul 31  2014 bzdiff

-rwxr-xr-x   1 root root        1677 Jul 31  2014 bzgrep

-rwxr-xr-x   1 root root       36752 Jul 31  2014 bzip2

-rwxr-xr-x   1 root root       15608 Jul 31  2014 bzip2recover

lrwxrwxrwx   1 root root           6 Jun 26  2017 bzless -> bzmore

-rwxr-xr-x   1 root root        1259 Jul 31  2014 bzmore

-rwxr-xr-x   4 root root      772704 Jun  2  2017 c++

-rwxr-xr-x   2 root root       36607 Mar  2  2017 c2ph

-rwxr-xr-x   1 root root         224 Jun  2  2017 c89

-rwxr-xr-x   1 root root         215 Jun  2  2017 c99

-rwxr-xr-x   1 root root       71600 Apr 19  2017 cairo-sphinx

-rwxr-xr-x   1 root root       33616 Jul 12  2018 cal

-rwxr-xr-x   1 root root        1638 May  5  2017 ca-legacy

-rwxr-xr-x   1 root root       25112 Jan 26 


-rwxr-xr-x   1 root root     4192008 Jul 26  2016 d8

-rwxr-xr-x   1 root root      117688 Jun 15  2017 danetool

-rwxr-xr-x   1 root root      153800 Feb 27  2018 dapltest

-rwxr-xr-x   1 root root       62200 Jun 30  2016 date

-rwxr-xr-x   1 root root       10456 Aug 19  2014 db47_archive

-rwxr-xr-x   1 root root       10520 Aug 19  2014 db47_checkpoint

-rwxr-xr-x   1 root root       22784 Aug 19  2014 db47_codegen

-rwxr-xr-x   1 root root       10528 Aug 19  2014 db47_deadlock

-rwxr-xr-x   1 root root       14592 Aug 19  2014 db47_dump

-rwxr-xr-x   1 root root       18816 Aug 19  2014 db47_hotbackup

-rwxr-xr-x   1 root root       26960 Aug 19  2014 db47_load

-rwxr-xr-x   1 root root       64304 Aug 19  2014 db47_printlog

-rwxr-xr-x   1 root root       10520 Aug 19  2014 db47_recover

-rwxr-xr-x   1 root root       14560 Aug 19  2014 db47_stat

-rwxr-xr-x   1 root root       10472 Aug 19  2014 db47_upgrade

-rwxr-xr-x   1 root root       10464 Aug 19  2014 db47_verify

-rwx


-rwxr-xr-x   1 root root        2911 May 30  2017 eu-make-debug-archive

-rwxr-xr-x   1 root root       45968 May 30  2017 eu-nm

-rwxr-xr-x   1 root root       33216 May 30  2017 eu-objdump

-rwxr-xr-x   1 root root       20048 May 30  2017 eu-ranlib

-rwxr-xr-x   1 root root      186520 May 30  2017 eu-readelf

-rwxr-xr-x   1 root root       19888 May 30  2017 eu-size

-rwxr-xr-x   1 root root       28304 May 30  2017 eu-stack

-rwxr-xr-x   1 root root       20064 May 30  2017 eu-strings

-rwxr-xr-x   1 root root       49800 May 30  2017 eu-strip

-rwxr-xr-x   1 root root       45272 May 30  2017 eu-unstrip

-rwxr-xr-x   1 root root      428504 Jul 17  2017 evince

-rwxr-xr-x   1 root root       50976 Jul 17  2017 evince-previewer

-rwxr-xr-x   1 root root       15824 Jul 17  2017 evince-thumbnailer

lrwxrwxrwx   1 root root           2 Sep 26  2017 ex -> vi

-rwxr-xr-x   1 root root       15848 Jan 24  2014 exempi

-rwxr-xr-x   1 root root       33240 Jun 30  2016 expand

-rwxr-xr-


-rwxr-xr-x   1 root root       11424 Feb 22  2017 getkeycodes

-rwxr-xr-x   1 root root       11320 Jan 27  2014 get_module

-rwxr-xr-x   1 root root       15832 Jul 12  2018 getopt

-rwxr-xr-x   1 root root          31 Aug  3  2017 getopts

-rwxr-xr-x   1 root root       36800 Mar 15  2017 gettext

-rwxr-xr-x   1 root root        4629 Mar 15  2017 gettext.sh

-rwxr-xr-x   1 root root        5126 Mar 30  2014 gflags_completions.sh

-rwxr-xr-x   1 root root      772688 Jun  2  2017 gfortran

lrwxrwxrwx   1 root root           2 Sep 26  2017 ghostscript -> gs

-rwxr-xr-x   1 root root       75240 Nov 10  2017 gio

-rwxr-xr-x   1 root root       11600 Nov 10  2017 gio-querymodules-64

-rwxr-xr-x   1 root root        2250 Feb  7  2017 g-ir-annotation-tool

-rwxr-xr-x   1 root root      176560 Feb  7  2017 g-ir-compiler

-rwxr-xr-x   1 root root        2229 Feb  7  2017 g-ir-doc-tool

-rwxr-xr-x   1 root root       37280 Feb  7  2017 g-ir-generate

-rwxr-xr-x   1 root root       11392 Feb 

...


-rwxr-xr-x   3 root root       76760 Feb 12  2014 rb

-rwxr-xr-x   1 root root     1315280 May  2  2016 rcc

-rwxr-xr-x   1 root root       15672 Feb 27  2018 rcopy

-rwxr-xr-x   1 root root        1707 Jun 21  2017 rct

-rwxr-xr-x   1 root root      152696 Sep 11  2017 rctest

-rwxr-xr-x   1 root root       10544 Jan 27  2014 rdate

-rwxr-xr-x   1 root root       41424 Jan 27  2014 rdfproc

-rwxr-xr-x   1 root root       11456 Feb 27  2018 rdma_client

-rwxr-xr-x   1 root root       11472 Feb 27  2018 rdma_server

-rwxr-xr-x   1 root root       11472 Feb 27  2018 rdma_xclient

-rwxr-xr-x   1 root root       11448 Feb 27  2018 rdma_xserver

-rwxr-xr-x   1 root root         934 Mar  6  2017 rdoc

-rwxr-xr-x   1 root root          28 Aug  3  2017 read

lrwxrwxrwx   1 root root          33 Jun 26  2017 readcd -> /etc/alternatives/cdrecord-readcd

-rwxr-xr-x   1 root root      493192 Aug  9  2017 readelf

-rwxr-xr-x   1 root root       41800 Jun 30  2016 readlink

-rwxr-xr-x   1 root root


-rwxr-xr-x   1 root root       20168 May 25  2017 seapplet

-rwxr-xr-x   1 root root       24648 May 26  2017 secon

-rwxr-xr-x   1 root root       20064 Mar 21  2017 secret-tool

-rwxr-xr-x   1 root root       76016 Feb 10  2014 sed

-rwxr-xr-x   1 root root      251192 Aug 11  2016 sedismod

-rwxr-xr-x   1 root root      180376 Aug 11  2016 sedispol

-rwxr-xr-x   1 root root       15656 May 26  2017 semodule_package

-rwxr-xr-x   1 root root       11824 Dec 28  2013 sendiso

-rwxr-xr-x   1 root root       49632 Jun 30  2016 seq

lrwxrwxrwx   1 root root          28 Sep 26  2017 servertool -> /etc/alternatives/servertool

-rwxr-xr-x   1 root root       11512 Feb  6  2017 sessreg

-rwxr-xr-x   1 root root       15712 Jul 12  2018 setarch

-rwxr-xr-x   1 root root       37616 Sep 21  2017 setfacl

-rwxr-xr-x   1 root root       16088 Sep 21  2017 setfattr

-rwxr-xr-x   1 root root       41344 Feb 22  2017 setfont

-rwxr-xr-x   1 root root       11424 Feb 22  2017 setkeycodes

-rwxr-xr-


-rwxr-xr-x   1 root root       83176 Aug 16  2017 systemd-firstboot

-rwxr-xr-x   1 root root       70720 Aug 16  2017 systemd-hwdb

-rwxr-xr-x   1 root root      275432 Aug 16  2017 systemd-inhibit

lrwxrwxrwx   1 root root           8 Sep 26  2017 systemd-loginctl -> loginctl

-rwxr-xr-x   1 root root       41248 Aug 16  2017 systemd-machine-id-setup

-rwxr-xr-x   1 root root       37032 Aug 16  2017 systemd-notify

-rwxr-xr-x   1 root root      483560 Aug 16  2017 systemd-nspawn

-rwxr-xr-x   1 root root       41256 Aug 16  2017 systemd-path

-rwxr-xr-x   1 root root      350160 Aug 16  2017 systemd-run

-rwxr-xr-x   1 root root      271328 Aug 16  2017 systemd-stdio-bridge

-rwxr-xr-x   1 root root        3979 Aug 16  2017 systemd-sysv-convert

-rwxr-xr-x   1 root root      120560 Aug 16  2017 systemd-tmpfiles

-rwxr-xr-x   1 root root       66192 Aug 16  2017 systemd-tty-ask-password-agent

-rwxr-xr-x   1 root root       24128 Jan 27  2014 systool

-rwxr-xr-x   3 root root       


-rwxr-xr-x   1 root root       11552 Jul 12  2018 uuidgen

-rwxr-xr-x   1 root root        3674 Jan 27  2014 uxterm

-rwxr-xr-x   1 root root        2307 Jan 26  2014 uz

lrwxrwxrwx   1 root root          22 Sep  7 18:27 vala -> /etc/alternatives/vala

lrwxrwxrwx   1 root root          10 Sep  7 18:27 vala-0.34 -> valac-0.34

lrwxrwxrwx   1 root root          23 Sep  7 18:27 valac -> /etc/alternatives/valac

-rwxr-xr-x   1 root root       41448 Mar  9  2017 valac-0.34

lrwxrwxrwx   1 root root          37 Sep  7 18:27 vala-gen-introspect -> /etc/alternatives/vala-gen-introspect

-rwxr-xr-x   1 root root         941 Mar  9  2017 vala-gen-introspect-0.34

lrwxrwxrwx   1 root root          27 Sep  7 18:27 vapicheck -> /etc/alternatives/vapicheck

-rwxr-xr-x   1 root root       70704 Mar  9  2017 vapicheck-0.34

lrwxrwxrwx   1 root root          25 Sep  7 18:27 vapigen -> /etc/alternatives/vapigen

-rwxr-xr-x   1 root root      175456 Mar  9  2017 vapigen-0.34

-rwxr-xr-x   1 root root   


-rwxr-xr-x   1 root root        1506 Dec 28  2013 xminicom

-rwxr-xr-x   1 root root        1716 Jun  6  2016 xml2-config

-rwxr-xr-x   1 root root       19752 Jun  6  2016 xmlcatalog

-rwxr-xr-x   1 root root       63408 Jun  6  2016 xmllint

-rwxr-xr-x   1 root root       71472 May  2  2016 xmlpatterns

-rwxr-xr-x   1 root root       15880 May  2  2016 xmlpatternsvalidator

-rwxr-xr-x   1 root root       61368 Aug  9  2017 xmlsec1

-rwxr-xr-x   1 root root       24568 Nov 24  2016 xmlwf

-rwxr-xr-x   1 root root       33056 Feb  6  2017 xmodmap

-rwxr-xr-x   1 root root        1063 Jul  2  2015 xmvn-builddep

-rwsr-xr-x   1 root root     2401104 Jun 26  2017 Xorg

-rwxr--r--   1 root root        1322 May 12  2015 xorg-x11-fonts-update-dirs

-rwxr-xr-x   1 root root       39248 Mar 28  2017 xprop

-rwxr-xr-x   1 root root       61512 Feb  6  2017 xrandr

-rwxr-xr-x   1 root root       28448 Feb  6  2017 xrdb

-rwxr-xr-x   1 root root       11424 Feb  6  2017 xrefresh

-rwxr-xr-x   1 

### `FILENAME`

If AWK is reading its input from a file specified on the command line, then this variable contains the name of the file.

### `FNR `

##### File record number

When reading input from a file specified on the command line, AWK sets this variable to the number of the record read from that file.

### Example: Filtering for American airports from `airports.dat`

Let's build an awk program that filters an airport database for all USA airports and aggregates them into an organized table.

In [9]:
# first examine the structure of the database

head airports.dat

1,"Goroka Airport","Goroka","Papua New Guinea","GKA","AYGA",-6.081689834590001,145.391998291,5282,10,"U","Pacific/Port_Moresby","airport","OurAirports"
2,"Madang Airport","Madang","Papua New Guinea","MAG","AYMD",-5.20707988739,145.789001465,20,10,"U","Pacific/Port_Moresby","airport","OurAirports"
3,"Mount Hagen Kagamuga Airport","Mount Hagen","Papua New Guinea","HGU","AYMH",-5.826789855957031,144.29600524902344,5388,10,"U","Pacific/Port_Moresby","airport","OurAirports"
4,"Nadzab Airport","Nadzab","Papua New Guinea","LAE","AYNZ",-6.569803,146.725977,239,10,"U","Pacific/Port_Moresby","airport","OurAirports"
5,"Port Moresby Jacksons International Airport","Port Moresby","Papua New Guinea","POM","AYPY",-9.443380355834961,147.22000122070312,146,10,"U","Pacific/Port_Moresby","airport","OurAirports"
6,"Wewak International Airport","Wewak","Papua New Guinea","WWK","AYWK",-3.58383011818,143.669006348,19,10,"U","Pacific/Port_Moresby","airport","OurAirports"
7,"Narsarsuaq Airport","Narssarssuaq",

For illustration purposes, let's report where each USA airport is located.

Example:

```
Dallas Love Field is located in Dallas
```

In [10]:
# Print a directory report

cat airports.dat | awk '
    
    # create header
    BEGIN { FS = ","
        print "USA Airports"
        print "================"
    }

    # relational expressions
    $4 == "\"United States\"" {
        gsub("\"","",$2) # built-in string function -- here we are stripping the quotes
        gsub("\"","",$3) # built-in string function -- here we are stripping the quotes
        print $2, "is located in", $3
    }

    # create footer
    END {
        print "============="
        print "End Of Report"
    }
'

USA Airports
Barter Island LRRS Airport is located in Barter Island
Wainwright Air Station is located in Fort Wainwright
Cape Lisburne LRRS Airport is located in Cape Lisburne
Point Lay LRRS Airport is located in Point Lay
Hilo International Airport is located in Hilo
Orlando Executive Airport is located in Orlando
Bettles Airport is located in Bettles
Clear Airport is located in Clear Mews
Indian Mountain LRRS Airport is located in Indian Mountains
Fort Yukon Airport is located in Fort Yukon
Sparrevohn LRRS Airport is located in Sparrevohn
Bryant Army Heliport is located in Fort Richardson
Tatalina LRRS Airport is located in Tatalina
Cape Romanzof LRRS Airport is located in Cape Romanzof
Laurence G Hanscom Field is located in Bedford
St Paul Island Airport is located in St. Paul Island
Cape Newenham LRRS Airport is located in Cape Newenham
St George Airport is located in Point Barrow
Iliamna Airport is located in Iliamna
Platinum Airport is located in Port Moller
Big Mountain Airport 

Warren Bud Woods Palmer Municipal Airport is located in Palmer
Pittsburgh International Airport is located in Pittsburgh
Wiley Post Will Rogers Memorial Airport is located in Barrow
Ellington Airport is located in Houston
Whidbey Island Naval Air Station (Ault Field) is located in Whidbey Island
Alice International Airport is located in Alice
Moody Air Force Base is located in Valdosta
Miami International Airport is located in Miami
Seattle Tacoma International Airport is located in Seattle
Lovell Field is located in Chattanooga
Igor I Sikorsky Memorial Airport is located in Stratford
Jackson-Medgar Wiley Evers International Airport is located in Jackson
Scholes International At Galveston Airport is located in Galveston
Long Beach /Daugherty Field/ Airport is located in Long Beach
Dillingham Airfield is located in Dillingham
Williamsport Regional Airport is located in Williamsport
Indianapolis International Airport is located in Indianapolis
Whiteman Air Force Base is located in Knobno

Palm Beach International Airport is located in West Palm Beach
Fort Worth Meacham International Airport is located in Fort Worth
Ogdensburg International Airport is located in Ogdensburg
Cape Cod Coast Guard Air Station is located in Falmouth
Boeing Field King County International Airport is located in Seattle
Lackland Air Force Base is located in San Antonio
Daniel K Inouye International Airport is located in Honolulu
Des Moines International Airport is located in Des Moines
Coastal Carolina Regional Airport is located in New Bern
San Diego International Airport is located in San Diego
Monroe Regional Airport is located in Monroe
Shaw Air Force Base is located in Sumter
Ontario International Airport is located in Ontario
Majors Airport is located in Greenvile
Roswell International Air Center Airport is located in Roswell
Coleman A. Young Municipal Airport is located in Detroit
Brownsville South Padre Island International Airport is located in Brownsville
Dothan Regional Airport is loc

Grissom Air Reserve Base is located in Peru
Casper-Natrona County International Airport is located in Casper
Destin-Ft Walton Beach Airport is located in Valparaiso
Craig Field is located in Selma
Key West International Airport is located in Key West
Charlotte Douglas International Airport is located in Charlotte
McCarran International Airport is located in Las Vegas
Orlando International Airport is located in Orlando
Florence Regional Airport is located in Florence
Great Falls International Airport is located in Great Falls
Youngstown Warren Regional Airport is located in Youngstown
Ladd AAF Airfield is located in Fort Wainwright
Mc Minnville Municipal Airport is located in Mackminnville
Robins Air Force Base is located in Macon
Pullman Moscow Regional Airport is located in Pullman
Lewiston Nez Perce County Airport is located in Lewiston
Elmira Corning Regional Airport is located in Elmira
Ithaca Tompkins Regional Airport is located in Ithaca
Monterey Peninsula Airport is located in M

Athens Ben Epps Airport is located in Athens
Alamogordo White Sands Regional Airport is located in Alamogordo
Waterloo Regional Airport is located in Waterloo
Walla Walla Regional Airport is located in Walla Walla
Alpena County Regional Airport is located in Alpena
Watertown Regional Airport is located in Watertown
Bradford Regional Airport is located in Bradford
Western Neb. Rgnl/William B. Heilig Airport is located in Scottsbluff
Raleigh County Memorial Airport is located in Beckley
Brunswick Golden Isles Airport is located in Brunswick
Southeast Iowa Regional Airport is located in Burlington
Jack Mc Namara Field Airport is located in Crescent City
Cape Girardeau Regional Airport is located in Cape Girardeau
Chippewa County International Airport is located in Sault Ste Marie
North Central West Virginia Airport is located in Clarksburg
William R Fairchild International Airport is located in Port Angeles
Houghton County Memorial Airport is located in Hancock
Dodge City Regional Airport

Ankeny Regional Airport is located in Ankeny
Corpus Christi Naval Air Station/Truax Field is located in Corpus Christi
Catalina Airport is located in Catalina Island
Mojave Airport is located in Mojave
Hutchinson Municipal Airport is located in Hutchinson
Rosecrans Memorial Airport is located in Rosecrans
Volk Field is located in Camp Douglas
Gunnison Crested Butte Regional Airport is located in Gunnison
Zamperini Field is located in Torrance
Manistee Co Blacker Airport is located in Manistee
Charlotte County Airport is located in Punta Gorda
Northern Aroostook Regional Airport is located in Frenchville
Chautauqua County-Jamestown Airport is located in Jamestown
Lake Cumberland Regional Airport is located in Somerset
Shenandoah Valley Regional Airport is located in Weyers Cave
Devils Lake Regional Airport is located in Devils Lake
Dickinson Theodore Roosevelt Regional Airport is located in Dickinson
Sidney - Richland Regional Airport is located in Sidney
Chadron Municipal Airport is lo

Indianola Municipal Airport is located in New York
French Valley Airport is located in Murrieta-Temecula
Carl R Keller Field is located in Port Clinton
Dayton-Wright Brothers Airport is located in Dayton
Richmond Municipal Airport is located in Richmond
Findlay Airport is located in Findley
Black Hills Airport-Clyde Ice Field is located in Spearfish-South Dakota
Olive Branch Airport is located in Olive Branch
Rocky Mountain Metropolitan Airport is located in Broomfield-CO
Salem Municipal Airport/McNary Field is located in Salem
Tunica Municipal Airport is located in Tunica
Lawrence J Timmerman Airport is located in Milwaukee
Southern Wisconsin Regional Airport is located in Janesville
Arlington Municipal Airport is located in Arlington
Gwinnett County Briscoe Field is located in Lawrenceville
Bowling Green Warren County Regional Airport is located in Bowling Green
Richard Lloyd Jones Jr Airport is located in Tulsa
Bryce Canyon Airport is located in Bryce Canyon
Burlington Alamance Regi

Ocala International Airport - Jim Taylor Field is located in Ocala
Jesup Wayne County Airport is located in Jesup
Madison Municipal Airport is located in Madison
Newnan Coweta County Airport is located in Newnan
Thomson-McDuffie County Airport is located in Thomson
Aiken Municipal Airport is located in Aiken
Woodward Field is located in Camden
Lumberton Regional Airport is located in Lumberton
Moore County Airport is located in Pinehurst-Southern Pines
Richmond County Airport is located in Rockingham
Baraboo Wisconsin Dells Airport is located in Baraboo
Statesville Regional Airport is located in Statesville
Burlington Municipal Airport is located in Burlington
William T. Piper Memorial Airport is located in Lock Haven
Zelienople Municipal Airport is located in Zelienople
Lorain County Regional Airport is located in Lorain-Elyria
Burke Lakefront Airport is located in Cleveland
Chautauqua County-Dunkirk Airport is located in Dunkirk
South Jersey Regional Airport is located in Mount Holly

Causey Airport is located in Liberty
Doylestown Airport is located in Doylestown
Warren Field is located in Washington
Hyde County Airport is located in Engelhard
Stillwater Regional Airport is located in Stillwater
Okmulgee Regional Airport is located in Okmulgee
Cushing Municipal Airport is located in Cushing
Clinton Sherman Airport is located in Clinton
Strother Field is located in Winfield
Wiley Post Airport is located in Oklahoma City
Shreveport Downtown Airport is located in Shreveport
Stephenville Clark Regional Airport is located in Stephenville
Perry Municipal Airport is located in Perry
Hamilton Municipal Airport is located in Hamilton
Ada Regional Airport is located in Ada
Mesquite Metro Airport is located in Misquite
Denton Municipal Airport is located in Denton
Austin Executive Airport is located in Austin
Lago Vista Tx Rusty Allen Airport is located in Lago Vista
Brenham Municipal Airport is located in Brenham
Lakeway Airpark is located in Lakeway
Iraan Municipal Airport 

Huntsville Regional Airport is located in Huntsville
Miller Field is located in Valentine
Winnemucca Municipal Airport is located in Winnemucca
West Woodward Airport is located in Woodward
Cape Canaveral AFS Skid Strip is located in Cocoa Beach
Homey (Area 51) Airport is located in Groom Lake
Zanesville Municipal Airport is located in Zanesville
Nenana Municipal Airport is located in Nenana
Wasilla Airport is located in Wasilla
Tracy Municipal Airport is located in Tracy
Bolton Field is located in Columbus
Skagit Regional Airport is located in Skagit
Marion County Brown Field is located in Jasper
Evergreen Regional Airport/Middleton Field is located in Evergreen
Pryor Field Regional Airport is located in Decatur
Glasgow Municipal Airport is located in Glasgow
Danielson Airport is located in Danielson
Clearfield Lawrence Airport is located in Clearfield
Jasper County Airport is located in Rensselaer
Statesboro Bulloch County Airport is located in Statesboro
New Smyrna Beach Municipal Ai

### More Complex Relational Expressions

Let's create a new report where only Canadian airports between 40 deg and 50 deg N latitude are listed.

Let's also format each record so that they appear like so:

```
Airport: ICAO Full Name
```

In [11]:
# Building a new database of Canadian airports between 40 N and 50 N latitude

# Print a directory report

cat airports.dat | awk '

    # create header
    BEGIN { FS = ","
        print "Canada Airports Between 40th and 50th parallels"
        print "==============================================="
    }

    # Three-conditions linked with pattern-logical operators
    $4 == "\"Canada\"" && $7 > 40 && $7 < 50 {
        gsub("\"","",$2)
        gsub("\"","",$6)
        print "Airport:", $6, $2 
    }

    # create footer
    END {
        print "==============================================="
        print "End Of Report"
    }
'

Canada Airports Between 40th and 50th parallels
Airport: CYAM Sault Ste Marie Airport
Airport: CYAW Halifax / CFB Shearwater Heliport
Airport: CYAZ Tofino / Long Beach Airport
Airport: CYBC Baie Comeau Airport
Airport: CYBG CFB Bagotville
Airport: CYBL Campbell River Airport
Airport: CYBR Brandon Municipal Airport
Airport: CYCD Nanaimo Airport
Airport: CYCG Castlegar/West Kootenay Regional Airport
Airport: CYCH Miramichi Airport
Airport: CYCL Charlo Airport
Airport: CYCW Chilliwack Airport
Airport: CYDC Princeton Airport
Airport: CYDF Deer Lake Airport
Airport: CYEN Estevan Airport
Airport: CYFC Fredericton Airport
Airport: CYFE Forestville Airport
Airport: CYGK Kingston Norman Rogers Airport
Airport: CYGP Gaspé (Michel-Pouliot) Airport
Airport: CYGQ Geraldton Greenstone Regional Airport
Airport: CYGR Îles-de-la-Madeleine Airport
Airport: CYHD Dryden Regional Airport
Airport: CYHM John C. Munro Hamilton International Airport
Airport: CYHU Montréal / Saint-Hubert Airport
Airport: CYHZ H

## Awk Programming
 
### User-defined functions

### Variables

##### Variables can be set in awk.

For example, we can treat a string of numbers as number by applying math to it.

```
deg = 46 + 0
min = 29 + 0
sec = 6 + 0
```

We can also force awk to treat a variable as a string by concatenating an empty string.

```
deg = 46 ""
```

### Arrays

Single-dimensional arrays are allowed in awk, and can be declared using the format:

```
array[key] = value
```

The key can be an integer or a string.

Example:

```
dms_lat["deg"] = 0 + 0
dms_lat["min"] = 0 + 0
dms_lat["sec"] = 0 + 0

dms_lon["deg"] = 0 + 0
dms_lon["min"] = 0 + 0
dms_lon["sec"] = 0 + 0
```

Arrays should be declared in the `BEGIN` pattern of your awk script.

```
BEGIN { FS = ","
    print "Canada Airports Between 40th and 50th parallels"
    print "==============================================="

    dms_lat["deg"] = 0 + 0
    dms_lat["min"] = 0 + 0
    dms_lat["sec"] = 0 + 0

    dms_lon["deg"] = 0 + 0
    dms_lon["min"] = 0 + 0
    dms_lon["sec"] = 0 + 0     
}
```

### User-defined functions

Awk allows for custom functions in a similar style to C.

Scalar-variables, strings and numbers, are passed by value, meaning they are copied into a local variable within the function.

Arrays are passed by reference, i.e. they act like global variables.

#### Example: A function that converts decimals coordinate to degrees-minutes-seconds

The scalar variables `coord_lat` and `coord_lon` are treated like local variables inside the function.

`dms_lat` is an existing array that is passed by reference and modified by the function.

```
function dec_dms(coord_lat, coord_lon) {
    dms_lat["deg"] = int(coord_lat)
    rem_deg = sqrt( (coord_lat - dms_lat["deg"])*(coord_lat - dms_lat["deg"]) )
    dms_lat["min"] = int(rem_deg*60)
    rem_min = rem_deg*60 - dms_lat["min"]
    dms_lat["sec"] = int(rem_min*60)

    dms_lon["deg"] = int(coord_lon)
    rem_deg = sqrt( (coord_lon - dms_lon["deg"])*(coord_lon - dms_lon["deg"]) )
    dms_lon["min"] = int(rem_deg*60)
    rem_min = rem_deg*60 - dms_lon["min"]
    dms_lon["sec"] = int(rem_min*60)        
}
```

### Example: Converting Airport Decimal Coordinates to Degrees-Minutes-Seconds

In [12]:
# Building a new database of Canadian airports between 40 N and 50 N latitude

# Print a directory report

cat airports.dat | awk '

    # We can define functions in Awk
    
    function dec_dms(coord_lat, coord_lon) {
        dms_lat["deg"] = int(coord_lat)
        rem_deg = sqrt( (coord_lat - dms_lat["deg"])*(coord_lat - dms_lat["deg"]) )
        dms_lat["min"] = int(rem_deg*60)
        rem_min = rem_deg*60 - dms_lat["min"]
        dms_lat["sec"] = int(rem_min*60)
        
        dms_lon["deg"] = int(coord_lon)
        rem_deg = sqrt( (coord_lon - dms_lon["deg"])*(coord_lon - dms_lon["deg"]) )
        dms_lon["min"] = int(rem_deg*60)
        rem_min = rem_deg*60 - dms_lon["min"]
        dms_lon["sec"] = int(rem_min*60)        
    }

    BEGIN { FS = ","
        print "Canada Airports Between 40th and 50th parallels"
        print "==============================================="
        
        # single dimensional arrays are allowed in AWK
        # Here we initialize an array for converting coordinates into deg-min-sec format
    
        dms_lat["deg"] = 0 + 0
        dms_lat["min"] = 0 + 0
        dms_lat["sec"] = 0 + 0

        dms_lon["deg"] = 0 + 0
        dms_lon["min"] = 0 + 0
        dms_lon["sec"] = 0 + 0     
    }

    $4 == "\"Canada\"" && $7 > 40 && $7 < 50 {
        gsub("\"","",$2)
        gsub("\"","",$6)
        dec_dms($7, $8)
        # print only the airport ICAO code and lat/long in DMS format
        print "Airport: ", $6, dms_lat["deg"] "\xc2\xb0", dms_lat["min"] "\x27", dms_lat["sec"] "\x22", dms_lon["deg"] "\xc2\xb0", dms_lon["min"] "\x27", dms_lon["sec"] "\x22"
    }

    END {
        print "==============================================="
        print "End Of Report"
    }
'

Canada Airports Between 40th and 50th parallels
Airport:  CYAM 46° 29' 6" -84° 30' 33"
Airport:  CYAW 44° 38' 22" -63° 29' 57"
Airport:  CYAZ 49° 4' 47" -125° 46' 32"
Airport:  CYBC 49° 7' 56" -68° 12' 15"
Airport:  CYBG 48° 19' 50" -70° 59' 47"
Airport:  CYBL 49° 57' 2" -125° 16' 15"
Airport:  CYBR 49° 54' 35" -99° 57' 6"
Airport:  CYCD 49° 3' 17" -123° 52' 11"
Airport:  CYCG 49° 17' 47" -117° 37' 55"
Airport:  CYCH 47° 0' 28" -65° 26' 57"
Airport:  CYCL 47° 59' 26" -66° 19' 49"
Airport:  CYCW 49° 9' 10" -121° 56' 20"
Airport:  CYDC 49° 28' 5" -120° 30' 39"
Airport:  CYDF 49° 12' 38" -57° 23' 29"
Airport:  CYEN 49° 12' 37" -102° 57' 57"
Airport:  CYFC 45° 52' 8" -66° 32' 13"
Airport:  CYFE 48° 44' 45" -69° 5' 49"
Airport:  CYGK 44° 13' 31" -76° 35' 48"
Airport:  CYGP 48° 46' 31" -64° 28' 42"
Airport:  CYGQ 49° 46' 41" -86° 56' 21"
Airport:  CYGR 47° 25' 28" -61° 46' 41"
Airport:  CYHD 49° 49' 54" -92° 44' 39"
Airport:  CYHM 43° 10' 24" -79° 56' 5"
Airport:  CYHU 45° 31' 3" -73° 25' 0"

### Expanding Our Example

##### Conditional statements

In [13]:
# Building a new database of Canadian airports between 40 N and 50 N latitude

# Print a directory report

cat airports.dat | awk '

    # We can also define functions in AWK
    
    function dec_dms(coord_lat, coord_lon) {
        dms_lat["deg"] = int(coord_lat)
        rem_deg = sqrt( (coord_lat - dms_lat["deg"])*(coord_lat - dms_lat["deg"]) )
        dms_lat["min"] = int(rem_deg*60)
        rem_min = rem_deg*60 - dms_lat["min"]
        dms_lat["sec"] = int(rem_min*60)

        # conditional statement to note which hemisphere
        if (coord_lat < 0) {
            dms_lat["hem"] = "S"
            dms_lat["deg"] = -1*dms_lat["deg"]
        }

        dms_lon["deg"] = int(coord_lon)
        rem_deg = sqrt( (coord_lon - dms_lon["deg"])*(coord_lon - dms_lon["deg"]) )
        dms_lon["min"] = int(rem_deg*60)
        rem_min = rem_deg*60 - dms_lon["min"]
        dms_lon["sec"] = int(rem_min*60)

        # conditional statement to note which hemisphere
        if (coord_lon < 0) {
            dms_lon["hem"] = "W"
            dms_lon["deg"] = -1*dms_lon["deg"]
        }
    }   

    BEGIN { FS = ","
        print "Canada Airports Between 40th and 50th parallels"
        print "==============================================="
        
        # single dimensional arrays are allowed in AWK
        # Here we initialize an array for converting coordinates into deg-min-sec format

        dms_lat["deg"] = 0 + 0
        dms_lat["min"] = 0 + 0
        dms_lat["sec"] = 0 + 0
        dms_lat["hem"] = "N"
        dms_lon["deg"] = 0 + 0
        dms_lon["min"] = 0 + 0
        dms_lon["sec"] = 0 + 0
        dms_lon["hem"] = "E"
    }

    $4 == "\"Canada\"" && $7 > 40 && $7 < 50 {
        gsub("\"","",$2)
        gsub("\"","",$6)
        dec_dms($7, $8)
        print "Airport: ", $6, dms_lat["deg"] "\xc2\xb0" dms_lat["hem"], dms_lat["min"] "\x27", dms_lat["sec"] "\x22", dms_lon["deg"] "\xc2\xb0" dms_lon["hem"], dms_lon["min"] "\x27", dms_lon["sec"] "\x22"
    }

    END {
        print "==============================================="
        print "End Of Report"
    }
'

Canada Airports Between 40th and 50th parallels
Airport:  CYAM 46°N 29' 6" 84°W 30' 33"
Airport:  CYAW 44°N 38' 22" 63°W 29' 57"
Airport:  CYAZ 49°N 4' 47" 125°W 46' 32"
Airport:  CYBC 49°N 7' 56" 68°W 12' 15"
Airport:  CYBG 48°N 19' 50" 70°W 59' 47"
Airport:  CYBL 49°N 57' 2" 125°W 16' 15"
Airport:  CYBR 49°N 54' 35" 99°W 57' 6"
Airport:  CYCD 49°N 3' 17" 123°W 52' 11"
Airport:  CYCG 49°N 17' 47" 117°W 37' 55"
Airport:  CYCH 47°N 0' 28" 65°W 26' 57"
Airport:  CYCL 47°N 59' 26" 66°W 19' 49"
Airport:  CYCW 49°N 9' 10" 121°W 56' 20"
Airport:  CYDC 49°N 28' 5" 120°W 30' 39"
Airport:  CYDF 49°N 12' 38" 57°W 23' 29"
Airport:  CYEN 49°N 12' 37" 102°W 57' 57"
Airport:  CYFC 45°N 52' 8" 66°W 32' 13"
Airport:  CYFE 48°N 44' 45" 69°W 5' 49"
Airport:  CYGK 44°N 13' 31" 76°W 35' 48"
Airport:  CYGP 48°N 46' 31" 64°W 28' 42"
Airport:  CYGQ 49°N 46' 41" 86°W 56' 21"
Airport:  CYGR 47°N 25' 28" 61°W 46' 41"
Airport:  CYHD 49°N 49' 54" 92°W 44' 39"
Airport:  CYHM 43°N 10' 24" 79°W 56' 5"
Airport:  CYHU

### Expanding Once More

##### printf "print format"

In [14]:
# Building a new database of Canadian airports between 40 N and 50 N latitude

# Print a directory report

cat airports.dat | awk '

    # We can also define functions in AWK
    
    function dec_dms(coord_lat, coord_lon) {
        dms_lat["deg"] = int(coord_lat)
        rem_deg = sqrt( (coord_lat - dms_lat["deg"])*(coord_lat - dms_lat["deg"]) )
        dms_lat["min"] = int(rem_deg*60)
        rem_min = rem_deg*60 - dms_lat["min"]
        dms_lat["sec"] = int(rem_min*60)

        # conditional statement to note which hemisphere
        if (coord_lat < 0) {
            dms_lat["hem"] = "S"
            dms_lat["deg"] = -1*dms_lat["deg"]
        }

        dms_lon["deg"] = int(coord_lon)
        rem_deg = sqrt( (coord_lon - dms_lon["deg"])*(coord_lon - dms_lon["deg"]) )
        dms_lon["min"] = int(rem_deg*60)
        rem_min = rem_deg*60 - dms_lon["min"]
        dms_lon["sec"] = int(rem_min*60)

        # conditional statement to note which hemisphere
        if (coord_lon < 0) {
            dms_lon["hem"] = "W"
            dms_lon["deg"] = -1*dms_lon["deg"]
        }
    }   

    BEGIN { FS = ","
        print "Canada Airports Between 40th and 50th parallels"
        print "==============================================="
        
        # single dimensional arrays are allowed in AWK
        # Here we initialize an array for converting coordinates into deg-min-sec format

        dms_lat["deg"] = 0
        dms_lat["min"] = 0
        dms_lat["sec"] = 0
        dms_lat["hem"] = "N"
        dms_lon["deg"] = 0
        dms_lon["min"] = 0
        dms_lon["sec"] = 0
        dms_lon["hem"] = "E"
    }

    $4 == "\"Canada\"" && $7 > 40 && $7 < 50 {
        gsub("\"","",$2)
        gsub("\"","",$6)
        dec_dms($7, $8)
        printf "%-5s %-5s %-4s %-4s %-5s %-4s %-4s \n", $6, dms_lat["deg"] "\xc2\xb0" dms_lat["hem"], dms_lat["min"] "\x27", dms_lat["sec"] "\x22", dms_lon["deg"] "\xc2\xb0" dms_lon["hem"], dms_lon["min"] "\x27", dms_lon["sec"] "\x22"
    }

    END {
        print "==============================================="
        print "End Of Report"
    }
'

Canada Airports Between 40th and 50th parallels
CYAM  46°N  29'  6"   84°W  30'  33"  
CYAW  44°N  38'  22"  63°W  29'  57"  
CYAZ  49°N  4'   47"  125°W 46'  32"  
CYBC  49°N  7'   56"  68°W  12'  15"  
CYBG  48°N  19'  50"  70°W  59'  47"  
CYBL  49°N  57'  2"   125°W 16'  15"  
CYBR  49°N  54'  35"  99°W  57'  6"   
CYCD  49°N  3'   17"  123°W 52'  11"  
CYCG  49°N  17'  47"  117°W 37'  55"  
CYCH  47°N  0'   28"  65°W  26'  57"  
CYCL  47°N  59'  26"  66°W  19'  49"  
CYCW  49°N  9'   10"  121°W 56'  20"  
CYDC  49°N  28'  5"   120°W 30'  39"  
CYDF  49°N  12'  38"  57°W  23'  29"  
CYEN  49°N  12'  37"  102°W 57'  57"  
CYFC  45°N  52'  8"   66°W  32'  13"  
CYFE  48°N  44'  45"  69°W  5'   49"  
CYGK  44°N  13'  31"  76°W  35'  48"  
CYGP  48°N  46'  31"  64°W  28'  42"  
CYGQ  49°N  46'  41"  86°W  56'  21"  
CYGR  47°N  25'  28"  61°W  46'  41"  
CYHD  49°N  49'  54"  92°W  44'  39"  
CYHM  43°N  10'  24"  79°W  56'  5"   
CYHU  45°N  31'  3"   73°W  25'  0"   
CYHZ  44°N  52' 

## Saving Our Script to An Awk Script File

In [15]:
echo '

    # We can also define functions in AWK
    
    function dec_dms(coord_lat, coord_lon) {
        dms_lat["deg"] = int(coord_lat)
        rem_deg = sqrt( (coord_lat - dms_lat["deg"])*(coord_lat - dms_lat["deg"]) )
        dms_lat["min"] = int(rem_deg*60)
        rem_min = rem_deg*60 - dms_lat["min"]
        dms_lat["sec"] = int(rem_min*60)

        # conditional statement to note which hemisphere
        if (coord_lat < 0) {
            dms_lat["hem"] = "S"
            dms_lat["deg"] = -1*dms_lat["deg"]
        }

        dms_lon["deg"] = int(coord_lon)
        rem_deg = sqrt( (coord_lon - dms_lon["deg"])*(coord_lon - dms_lon["deg"]) )
        dms_lon["min"] = int(rem_deg*60)
        rem_min = rem_deg*60 - dms_lon["min"]
        dms_lon["sec"] = int(rem_min*60)

        # conditional statement to note which hemisphere
        if (coord_lon < 0) {
            dms_lon["hem"] = "W"
            dms_lon["deg"] = -1*dms_lon["deg"]
        }
    }   

    BEGIN { FS = ","
        print "Canada Airports Between 40th and 50th parallels"
        print "==============================================="
        
        # single dimensional arrays are allowed in AWK
        # Here we initialize an array for converting coordinates into deg-min-sec format

        dms_lat["deg"] = 0
        dms_lat["min"] = 0
        dms_lat["sec"] = 0
        dms_lat["hem"] = "N"
        dms_lon["deg"] = 0
        dms_lon["min"] = 0
        dms_lon["sec"] = 0
        dms_lon["hem"] = "E"
    }

    $4 == "\"Canada\"" && $7 > 40 && $7 < 50 {
        gsub("\"","",$2)
        gsub("\"","",$6)
        dec_dms($7, $8)
        printf "%-5s %-5s %-4s %-4s %-5s %-4s %-4s \n", $6, dms_lat["deg"] "\xc2\xb0" dms_lat["hem"], dms_lat["min"] "\x27", dms_lat["sec"] "\x22", dms_lon["deg"] "\xc2\xb0" dms_lon["hem"], dms_lon["min"] "\x27", dms_lon["sec"] "\x22"
    }

    END {
        print "==============================================="
        print "End Of Report"
    }
' > awk_prog.awk

In [16]:
cat awk_prog.awk



    # We can also define functions in AWK
    
    function dec_dms(coord_lat, coord_lon) {
        dms_lat["deg"] = int(coord_lat)
        rem_deg = sqrt( (coord_lat - dms_lat["deg"])*(coord_lat - dms_lat["deg"]) )
        dms_lat["min"] = int(rem_deg*60)
        rem_min = rem_deg*60 - dms_lat["min"]
        dms_lat["sec"] = int(rem_min*60)

        # conditional statement to note which hemisphere
        if (coord_lat < 0) {
            dms_lat["hem"] = "S"
            dms_lat["deg"] = -1*dms_lat["deg"]
        }

        dms_lon["deg"] = int(coord_lon)
        rem_deg = sqrt( (coord_lon - dms_lon["deg"])*(coord_lon - dms_lon["deg"]) )
        dms_lon["min"] = int(rem_deg*60)
        rem_min = rem_deg*60 - dms_lon["min"]
        dms_lon["sec"] = int(rem_min*60)

        # conditional statement to note which hemisphere
        if (coord_lon < 0) {
            dms_lon["hem"] = "W"
            dms_lon["deg"] = -1*dms_lon["deg"]
        }
    }   

    BEGIN { FS = ","
        print "C

In [17]:
cat airports.dat | awk -f awk_prog.awk

Canada Airports Between 40th and 50th parallels
CYAM  46°N  29'  6"   84°W  30'  33"  
CYAW  44°N  38'  22"  63°W  29'  57"  
CYAZ  49°N  4'   47"  125°W 46'  32"  
CYBC  49°N  7'   56"  68°W  12'  15"  
CYBG  48°N  19'  50"  70°W  59'  47"  
CYBL  49°N  57'  2"   125°W 16'  15"  
CYBR  49°N  54'  35"  99°W  57'  6"   
CYCD  49°N  3'   17"  123°W 52'  11"  
CYCG  49°N  17'  47"  117°W 37'  55"  
CYCH  47°N  0'   28"  65°W  26'  57"  
CYCL  47°N  59'  26"  66°W  19'  49"  
CYCW  49°N  9'   10"  121°W 56'  20"  
CYDC  49°N  28'  5"   120°W 30'  39"  
CYDF  49°N  12'  38"  57°W  23'  29"  
CYEN  49°N  12'  37"  102°W 57'  57"  
CYFC  45°N  52'  8"   66°W  32'  13"  
CYFE  48°N  44'  45"  69°W  5'   49"  
CYGK  44°N  13'  31"  76°W  35'  48"  
CYGP  48°N  46'  31"  64°W  28'  42"  
CYGQ  49°N  46'  41"  86°W  56'  21"  
CYGR  47°N  25'  28"  61°W  46'  41"  
CYHD  49°N  49'  54"  92°W  44'  39"  
CYHM  43°N  10'  24"  79°W  56'  5"   
CYHU  45°N  31'  3"   73°W  25'  0"   
CYHZ  44°N  52' 